# Implementation of TextRank
(Based on: https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

The input text is given below

In [1]:
#Source of text:
#https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents

Text = u"""
    Одной из популярных позиций нашего производства является угловой диван Респект. Если Вы современный и деловой человек, любите комфорт и уют, а каждый сантиметр жилплощади для Вас на вес золота – тогда угловой диван Респект отлично впишется в интерьер Вашей квартиры, заняв при этом минимум места. Угловой диван Респект не только практичен в использовании, удобен, мягок и современен, но и еще удивительно компактен. В его изготовлении используются только самые экологически чистые ткани и наполнители, а механизм раскладки еврокнижка - поможет Вам сэкономить массу усилий и времени для его преобразования в полноценную кровать.
     Модель со схемой раскладки еврокнижка, сделана на основе деревянного каркаса с использованием пружины типа Боннель в паре с пружинной змейкой.
Особенностью этого дивана является небольшой габаритный размер короткой части модели 135см, что позволяет устанавливать его в наши малогабаритные квартиры. Стильные боковые перила в виде полудуги придают угловому дивану Респект чрезвычайной элегантности.
     Как и в большинстве моделей нашего производства в модели Респект есть возможность добавления на спальную и сидячую части натуральных наполнителей (кокос, латекс, спрут, шерсть и др.), что придаст модели более выраженный ортопедический эффект. В конструкцию углового дивана заложено два больших места для хранения постельного белья, одно под сидячей продольной частью, второе в угловой части. Угловой диван Респект комплектуется тремя большими подушками для удобного сидения в собранном виде.
     Все материалы, которые используются при производстве углового дивана Респект проверены и сертифицированы, на весь модельный ряд и в частности на эту модель распространяется гарантийное и послегарантийное обслуживание.
    """

In [32]:
try:
    nltk.data.find('averaged_perceptron_tagger_ru')
except LookupError:
    print('bad')

bad


### Cleaning Text Data

The raw input text is cleaned off non-printable characters (if any) and turned into lower case.
The processed input text is then tokenized using NLTK library functions. 

In [33]:
import re
import nltk
from nltk import word_tokenize, RegexpTokenizer
import string

#nltk.download('punkt')
cyrrilic = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
def clean(text):
    text = text.lower()
    printable = set(cyrrilic + string.printable)
#     print(printable)
    text = re.sub("[^{}]".format(printable), '', text) #filter funny characters, if any.
    return text

Cleaned_text = clean(Text)
tokenizer = RegexpTokenizer(r'\w+')
text = tokenizer.tokenize(Cleaned_text)

print( "Tokenized Text: \n")
# print (text)

Tokenized Text: 



### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

Description of POS tags: 

- English
http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

- Russian (from Russian National Corpus tagset)
http://www.ruscorpora.ru/en/corpora-morph.html

In [3]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/h.pylieva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [40]:
# #nltk.download('averaged_perceptron_tagger')
  
# POS_tag = nltk.pos_tag(text, lang='rus')

# print("Tokenized Text with POS tags: \n")
# print(POS_tag)

### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

Details about lemmatization: 
    
https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [4]:
#nltk.download('wordnet')




# adj_type = ['A','A-NUM']
# spec = ['m', 'f', 'pl', 'n']
# adjective_tags = ['='.join([t,s]) for t in adj_type for s in spec]
# for English version ['JJ','JJR','JJS']

# lemmatized_text = []

# for word in POS_tag:
#     if word[1] in adjective_tags:
#         lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
#     else:
#         lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
lemmatized_text =  [morph_ru.parse(word)[0].normal_form for word in text]        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
# print (lemmatized_text)

Text tokens after lemmatization of adjectives and nouns: 



### POS tagging for Filtering

The <b>lemmatized text</b> is <b>POS tagged</b> here. The tags will be used for filtering later on.

In [5]:
POS_tag = nltk.pos_tag(lemmatized_text, lang='rus')

print ("Lemmatized text with POS tags: \n")
# print (POS_tag)

Lemmatized text with POS tags: 



In [43]:
np.unique(list(zip(*POS_tag))[1])

array(['A-PRO=m', 'A-PRO=pl', 'A=f', 'A=m', 'ADV', 'ADV-PRO', 'ANUM=f',
       'CONJ', 'NUM=acc', 'NUM=ciph', 'NUM=m', 'PART', 'PR', 'S', 'S-PRO',
       'V'], dtype='<U8')

## POS Based Filtering

Any word from the lemmatized text, which isn't a noun, adjective, or gerund (or a 'foreign word'), is here
considered as a <b>stopword</b> (non-content). This is based on the assumption that usually keywords are noun,
adjectives or gerunds. 

Punctuations are added to the stopword list too.

In [7]:
import pymorphy2

stopwords = []

# adj_type = ['A', 'A-PRO']
# spec = ['m', 'f', 'pl', 'n']
# adjective_tags = ['='.join([t,s]) for t in adj_type for s in spec]
adjective_tags = ['A=m', 'A=f', 'A=pl', 'A=n', 'A-NUM']

wanted_POS = ['S',
#               'V'
             ] + adjective_tags
# for English ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

### Complete stopword generation

Even if we remove the aforementioned stopwords, still some extremely common nouns, adjectives or gerunds may
remain which are very bad candidates for being keywords (or part of it). 

An external file constituting a long list of stopwords is loaded and all the words are added with the previous
stopwords to create the final list 'stopwords-plus' which is then converted into a set. 

(Source of stopwords data: https://www.ranks.nl/stopwords)

Stopwords-plus constitute the sum total of all stopwords and potential phrase-delimiters. 

(The contents of this set will be later used to partition the lemmatized text into n-gram phrases. But, for now, I will simply remove the stopwords, and work with a 'bag-of-words' approach. I will be developing the graph using unigram texts as vertices)

In [9]:
stopword_file = open("long_stopwords_ru.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

### Removing Stopwords 

Removing stopwords from lemmatized_text. 
Processeced_text condtains the result.

In [10]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print (processed_text)

['популярный', 'позиция', 'производство', 'угловой', 'диван', 'респект', 'современный', 'деловой', 'комфорт', 'уют', 'сантиметр', 'жилплощадь', 'вес', 'золото', 'угловой', 'диван', 'респект', 'интерьер', 'квартира', 'минимум', 'место', 'угловой', 'диван', 'респект', 'использование', 'удобный', 'мягкий', 'современный', 'компактный', 'изготовление', 'чистый', 'ткань', 'наполнитель', 'механизм', 'раскладка', 'еврокнижка', 'масса', 'усилие', 'преобразование', 'полноценный', 'кровать', 'модель', 'схема', 'раскладка', 'еврокнижка', 'основа', 'деревянный', 'каркас', 'использование', 'пружина', 'тип', 'боннель', 'пар', 'пружинный', 'змейка', 'особенность', 'диван', 'небольшой', 'габаритный', 'размер', 'короткий', 'часть', 'модель', 'малогабаритный', 'квартира', 'стильный', 'боковой', 'вид', 'полудуга', 'угловой', 'диван', 'респект', 'чрезвычайный', 'элегантность', 'большинство', 'модель', 'производство', 'модель', 'респект', 'возможность', 'добавление', 'спальная', 'сидячий', 'часть', 'натурал

## Vocabulary Creation

Vocabulary will only contain unique words from processed_text.

In [11]:
vocabulary = list(set(processed_text))
print (vocabulary)

['больший', 'сантиметр', 'популярный', 'использование', 'латекс', 'масса', 'усилие', 'элегантность', 'возможность', 'размер', 'место', 'собранный', 'послегарантийный', 'мягкий', 'компактный', 'жилплощадь', 'современный', 'производство', 'пружинный', 'вид', 'еврокнижка', 'чрезвычайный', 'продольный', 'модельный', 'обслуживание', 'эффект', 'ткань', 'основа', 'конструкция', 'габаритный', 'тип', 'гарантийный', 'изготовление', 'золото', 'респект', 'чистый', 'полудуга', 'боковой', 'удобный', 'комфорт', 'схема', 'стильный', 'пар', 'минимум', 'пружина', 'короткий', 'сидячий', 'раскладка', 'шерсть', 'постельный', 'материал', 'вес', 'ортопедический', 'деревянный', 'небольшой', 'модель', 'бельё', 'наполнитель', 'добавление', 'уют', 'натуральный', 'диван', 'механизм', 'деловой', 'особенность', 'часть', 'сидение', 'преобразование', 'хранение', 'спальная', 'позиция', 'большинство', 'ряд', 'каркас', 'интерьер', 'боннель', 'полноценный', 'кровать', 'кокос', 'подушка', 'частность', 'квартира', 'малогаб

### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [12]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


### Calculating weighted summation of connections of a vertex

inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.

In [13]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

### Scoring Vertices

The formula used for scoring a vertex represented by i is:

score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i. 

d is the damping factor.

The score is iteratively updated until convergence. 

In [14]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print( "Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 29....


In [15]:
for i in range(0,vocab_len):
    print ("Score of "+vocabulary[i]+": "+str(score[i]))

Score of больший: 1.1516752
Score of сантиметр: 0.85898554
Score of популярный: 0.45273048
Score of использование: 1.356987
Score of латекс: 0.7797809
Score of масса: 0.8131534
Score of усилие: 0.83935505
Score of элегантность: 0.70660686
Score of возможность: 0.6972453
Score of размер: 0.74894476
Score of место: 1.1955262
Score of собранный: 0.7303076
Score of послегарантийный: 0.47902593
Score of мягкий: 0.73600566
Score of компактный: 0.7962594
Score of жилплощадь: 0.8380453
Score of современный: 1.3552414
Score of производство: 1.7572266
Score of пружинный: 0.8548434
Score of вид: 1.3042666
Score of еврокнижка: 1.4513521
Score of чрезвычайный: 0.6755964
Score of продольный: 0.6805772
Score of модельный: 0.6859055
Score of обслуживание: 0.15
Score of эффект: 0.71281666
Score of ткань: 0.8135988
Score of основа: 0.7951499
Score of конструкция: 0.677244
Score of габаритный: 0.74183095
Score of тип: 0.85069686
Score of гарантийный: 0.70578146
Score of изготовление: 0.82140166
Score of 

### Phrase Partiotioning

Paritioning lemmatized_text into phrases using the stopwords in it as delimeters.
The phrases are also candidates for keyphrases to be extracted. 

In [17]:
phrases = []

phrase = " "
for word in lemmatized_text:
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print("Partitioned Phrases (Candidate Keyphrases): \n")
print(phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['популярный', 'позиция'], ['производство'], ['угловой', 'диван', 'респект'], ['современный'], ['деловой'], ['комфорт'], ['уют'], ['сантиметр', 'жилплощадь'], ['вес', 'золото'], ['угловой', 'диван', 'респект'], ['интерьер'], ['квартира'], ['минимум', 'место', 'угловой', 'диван', 'респект'], ['использование', 'удобный', 'мягкий'], ['современный'], ['компактный'], ['изготовление'], ['чистый', 'ткань'], ['наполнитель'], ['механизм', 'раскладка', 'еврокнижка'], ['масса', 'усилие'], ['преобразование'], ['полноценный', 'кровать', 'модель'], ['схема', 'раскладка', 'еврокнижка'], ['основа', 'деревянный', 'каркас'], ['использование', 'пружина', 'тип', 'боннель'], ['пар'], ['пружинный', 'змейка', 'особенность'], ['диван'], ['небольшой', 'габаритный', 'размер', 'короткий', 'часть', 'модель'], ['малогабаритный', 'квартира', 'стильный', 'боковой'], ['вид', 'полудуга'], ['угловой', 'диван', 'респект', 'чрезвычайный', 'элегантность'], ['большинство', 'мо

### Create a list of unique phrases.

Repeating phrases\keyphrase candidates has no purpose here, anymore. 

In [18]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print("Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['популярный', 'позиция'], ['производство'], ['угловой', 'диван', 'респект'], ['современный'], ['деловой'], ['комфорт'], ['уют'], ['сантиметр', 'жилплощадь'], ['вес', 'золото'], ['интерьер'], ['квартира'], ['минимум', 'место', 'угловой', 'диван', 'респект'], ['использование', 'удобный', 'мягкий'], ['компактный'], ['изготовление'], ['чистый', 'ткань'], ['наполнитель'], ['механизм', 'раскладка', 'еврокнижка'], ['масса', 'усилие'], ['преобразование'], ['полноценный', 'кровать', 'модель'], ['схема', 'раскладка', 'еврокнижка'], ['основа', 'деревянный', 'каркас'], ['использование', 'пружина', 'тип', 'боннель'], ['пар'], ['пружинный', 'змейка', 'особенность'], ['диван'], ['небольшой', 'габаритный', 'размер', 'короткий', 'часть', 'модель'], ['малогабаритный', 'квартира', 'стильный', 'боковой'], ['вид', 'полудуга'], ['угловой', 'диван', 'респект', 'чрезвычайный', 'элегантность'], ['большинство', 'модель'], ['модель', 'респект'], ['возможность', 'добавле

### Thinning the list of candidate-keyphrases.

Removing single word keyphrases-candidates that are present multi-word alternatives. 

In [23]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print( "Thinned Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases)    

Thinned Unique Phrases (Candidate Keyphrases): 

[['популярный', 'позиция'], ['угловой', 'диван', 'респект'], ['современный'], ['деловой'], ['комфорт'], ['уют'], ['сантиметр', 'жилплощадь'], ['вес', 'золото'], ['интерьер'], ['минимум', 'место', 'угловой', 'диван', 'респект'], ['использование', 'удобный', 'мягкий'], ['компактный'], ['изготовление'], ['чистый', 'ткань'], ['механизм', 'раскладка', 'еврокнижка'], ['масса', 'усилие'], ['преобразование'], ['полноценный', 'кровать', 'модель'], ['схема', 'раскладка', 'еврокнижка'], ['основа', 'деревянный', 'каркас'], ['использование', 'пружина', 'тип', 'боннель'], ['пар'], ['пружинный', 'змейка', 'особенность'], ['небольшой', 'габаритный', 'размер', 'короткий', 'часть', 'модель'], ['малогабаритный', 'квартира', 'стильный', 'боковой'], ['вид', 'полудуга'], ['угловой', 'диван', 'респект', 'чрезвычайный', 'элегантность'], ['большинство', 'модель'], ['модель', 'респект'], ['возможность', 'добавление'], ['спальная'], ['сидячий', 'часть', 'натуральн

### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [24]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: 'популярный позиция', Score: 1.099055826663971
Keyword: 'угловой диван респект', Score: 11.897812128067017
Keyword: 'современный', Score: 1.3552414178848267
Keyword: 'деловой', Score: 0.7743292450904846
Keyword: 'комфорт', Score: 0.8298757672309875
Keyword: 'уют', Score: 0.8569303750991821
Keyword: 'сантиметр жилплощадь', Score: 1.6970308423042297
Keyword: 'вес золото', Score: 1.497110664844513
Keyword: 'интерьер', Score: 0.6507393717765808
Keyword: 'минимум место угловой диван респект', Score: 13.75980293750763
Keyword: 'использование удобный мягкий', Score: 3.39951229095459
Keyword: 'компактный', Score: 0.7962594032287598
Keyword: 'изготовление', Score: 0.8214016556739807
Keyword: 'чистый ткань', Score: 1.6413831114768982
Keyword: 'механизм раскладка еврокнижка', Score: 3.63089519739151
Keyword: 'масса усилие', Score: 1.6525084376335144
Keyword: 'преобразование', Score: 0.8467838168144226
Keyword: 'полноценный кровать модель', Score: 5.168544411659241
Keyword: 'схема расклад

In [27]:
filtered_keywords = [kw for kw in keywords  if len(kw.split())<=4]

In [28]:
filtered_keywords

['популярный позиция',
 'угловой диван респект',
 'современный',
 'деловой',
 'комфорт',
 'уют',
 'сантиметр жилплощадь',
 'вес золото',
 'интерьер',
 'использование удобный мягкий',
 'компактный',
 'изготовление',
 'чистый ткань',
 'механизм раскладка еврокнижка',
 'масса усилие',
 'преобразование',
 'полноценный кровать модель',
 'схема раскладка еврокнижка',
 'основа деревянный каркас',
 'использование пружина тип боннель',
 'пар',
 'пружинный змейка особенность',
 'малогабаритный квартира стильный боковой',
 'вид полудуга',
 'большинство модель',
 'модель респект',
 'возможность добавление',
 'спальная',
 'шерсть',
 'ортопедический эффект',
 'конструкция угловой диван',
 'больший место',
 'хранение постельный бельё',
 'сидячий продольный часть',
 'больший подушка',
 'удобный сидение',
 'собранный вид',
 'материал',
 'производство угловой диван респект',
 'модельный ряд',
 'частность',
 'гарантийный']

### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [21]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 10

print ("Keywords:\n")

for i in range(0,keywords_num):
    print (str(keywords[sorted_index[i]])+", ")

Keywords:

угловой часть угловой диван респект, 
минимум место угловой диван респект, 
производство угловой диван респект, 
угловой диван респект чрезвычайный элегантность, 
угловой диван респект, 
конструкция угловой диван, 
небольшой габаритный размер короткий часть модель, 
модель респект, 
сидячий часть натуральный наполнитель кокос латекс, 
полноценный кровать модель, 


# Input:

Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types of systems and systems of mixed types.

# Extracted Keywords:

* minimal supporting set,  
* minimal generating set,  
* minimal set,  
* linear diophantine equation,  
* nonstrict inequations,  
* strict inequations,  
* system,  
* linear constraint,  
* solution,  
* upper bound, 
